In [1]:
import os
import sys
import nashpy
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt

import multiprocessing
from joblib import delayed
from joblib import Parallel

In [2]:
# ASSUMPTIONS
C = 1000
w = 0.2
gm = 0.02
v0 = 130

In [4]:
def payoff_attack(rate, tau, C, o, g, v0, v1, lbd = 50):
    
    if stats.poisson.cdf(k = tau, mu = lbd) > 0.5:
        rt = rate
    else:
        rt = 0
    
    part1 = ((tau*rt)/(C**2)) - o - g
    delta_v = ((v0 - v1) / v0)#*1000
    
    # Scaling
    minimum = max(part1, delta_v)
    maximum = min(part1, delta_v)
    
    part1 = (part1 - minimum)/(maximum - minimum)
    delta_v = (delta_v - minimum)/(maximum - minimum)
    
    return part1 + delta_v

payoff_attack = np.vectorize(payoff_attack)#, excluded = {"rate", "C", "o", "g", "v0", "v1", "lbd"})

In [34]:
def payoff_defense(rate, tau, C, v0, v1, lbd = 50):
    
    if stats.poisson.cdf(k = tau, mu = lbd) > 0.5:
        rt = rate
    else:
        rt = 0
    
    delta_v = ((v0 - v1) / v0)#*1000
    part1 = 1 - ((tau*rt)/(C**2))
    
    # Scaling
    #minimum = max(part1, delta_v)
    #maximum = min(part1, delta_v)
    
    #part1 = (part1 - minimum)/(maximum - minimum)
    #delta_v = (delta_v - minimum)/(maximum - minimum)    
    
    return part1 - delta_v

payoff_defense = np.vectorize(payoff_defense)

In [35]:
def Simulation(C, omega, gamma, v0, rate_list = None,
               tau_list = None, v1_list = None, lbd = 50):
    """
    Parameters
    ----------
    
    rate_list : iterable
    tau_list : iterable
    v1_list : iterable
    """
    output_payoff = []
    output_strategy = []
    
    if tau_list is None:
        tau_list = np.arange(200, 1001, 100)
    if rate_list is None:
        rate_list = np.arange(200, 1001, 100)
    if v1_list is None:
        v1_list = np.arange(v0 - 30, v0 + 30, 10)
    
    for v1 in v1_list:

        ATK = np.array([payoff_attack(rate = r, tau = tau_list, C = C, o = omega,
                                      g = gamma, v0 = v0, v1 = v1, lbd = lbd)
                        for r in rate_list])
        DEF = np.array([payoff_defense(rate = r, tau = tau_list, C = C,
                                       v0 = v0, v1 = v1, lbd = lbd)
                        for r in rate_list])

        game = nashpy.Game(ATK, DEF)
        
        equilibria = []
        for eq in game.support_enumeration():
            
            iter_ = eq
            
            br_player1 = iter_[0].argmax()
            br_player2 = iter_[1].argmax()
            
            # Payoffs
            output_payoff += [[v1,
                               ATK[br_player1, br_player2],
                               DEF[br_player1, br_player2]]]            
            # Strategies
            output_strategy += [[v1, rate_list[br_player1], tau_list[br_player2]]]
            
    return output_payoff, output_strategy

In [61]:
stats.poisson.cdf(k = 200, mu = 200)

0.5187943096786846

In [39]:
n_cores = min(multiprocessing.cpu_count(), 8) # don't use more than 8 cores

prova = Parallel(n_jobs = n_cores)(delayed(lambda v:
                                           Simulation(C = C, omega = w, gamma = gm, v0 = v0,
                                                      rate_list = np.arange(200, 1001, 100),
                                                      tau_list = np.arange(500, 1001, 100),
                                                      v1_list = [v], lbd = 50))(v)
                                           for v in [10, 20, 30, 40, 50])

o_pfs = [prova[i][0] for i in range(len(prova))]
o_str = [prova[i][1] for i in range(len(prova))]

print(o_pfs)
print(o_str)

[[[10, 1.0, -0.023076923076923106], [10, 1.0, -0.07307692307692315], [10, 1.0, -0.12307692307692308], [10, 1.0, -0.17307692307692313], [10, 1.0, -0.22307692307692317], [10, 1.0, -0.2730769230769231], [10, 1.0, -0.32307692307692315], [10, 1.0, -0.3730769230769231], [10, 1.0, -0.42307692307692313]], [[20, 1.0, 0.05384615384615388], [20, 1.0, 0.0038461538461538325], [20, 1.0, -0.0461538461538461], [20, 1.0, -0.09615384615384615], [20, 1.0, -0.1461538461538462], [20, 1.0, -0.19615384615384612], [20, 1.0, -0.24615384615384617], [20, 1.0, -0.2961538461538461], [20, 1.0, -0.34615384615384615]], [[30, 1.0, 0.13076923076923075], [30, 1.0, 0.0807692307692307], [30, 1.0, 0.03076923076923077], [30, 1.0, -0.019230769230769273], [30, 1.0, -0.06923076923076932], [30, 1.0, -0.11923076923076925], [30, 1.0, -0.1692307692307693], [30, 1.0, -0.21923076923076923], [30, 1.0, -0.2692307692307693]], [[40, 1.0, 0.20769230769230773], [40, 1.0, 0.1576923076923077], [40, 1.0, 0.10769230769230775], [40, 1.0, 0.057

In [37]:
pd.DataFrame(np.vstack([np.array(c) for c in o_str])).shape

(45, 3)

---
---
---